In [ ]:
import os
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import streamlit as st

# Télécharger les ressources nécessaires pour NLTK
nltk.download('stopwords')  # Télécharge les mots vides pour la langue anglaise
nltk.download('wordnet')  # Télécharge le dictionnaire WordNet pour la lemmatisation

# Constantes
OUTPUT_DIR = "spam_classifier_output"  # Dossier pour sauvegarder les fichiers de sortie
MODEL_PATH = os.path.join(OUTPUT_DIR, "spam_classifier_model.pkl")  # Chemin du fichier modèle
VECTORIZER_PATH = os.path.join(OUTPUT_DIR, "tfidf_vectorizer.pkl")  # Chemin du fichier vectoriseur
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Crée le dossier de sortie s'il n'existe pas

# Nettoyage des données
def clean_text(text: str) -> str:
    """Nettoie le texte pour la classification."""
    if not isinstance(text, str):
        return ""  # Retourne une chaîne vide si l'entrée n'est pas une chaîne
    text = text.lower()  # Convertit le texte en minuscule
    text = re.sub(r'http\S+|www\S+', 'url', text)  # Remplace les URLs par 'url'
    text = re.sub(r'[^a-z\s]', '', text)  # Supprime les caractères non alphabétiques
    stop_words = set(stopwords.words('english'))  # Liste des mots vides en anglais
    lemmatizer = WordNetLemmatizer()  # Initialisation du lemmatiseur
    return ' '.join(
        lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words
    )  # Lemmatisation et suppression des mots vides

# Charger ou sauvegarder le modèle et le vectoriseur
def load_model_and_vectorizer():
    """Charge le modèle et le vectoriseur à partir des fichiers."""
    try:
        if os.path.exists(MODEL_PATH) and os.path.exists(VECTORIZER_PATH):
            model = joblib.load(MODEL_PATH)  # Charge le modèle sauvegardé
            vectorizer = joblib.load(VECTORIZER_PATH)  # Charge le vectoriseur sauvegardé
            return model, vectorizer
    except Exception as e:
        st.error(f"Erreur lors du chargement : {e}")  # Affiche une erreur dans l'interface Streamlit
    return None, None  # Retourne None si le chargement échoue

def save_model_and_vectorizer(model, vectorizer):
    """Sauvegarde le modèle et le vectoriseur."""
    joblib.dump(model, MODEL_PATH)  # Sauvegarde le modèle
    joblib.dump(vectorizer, VECTORIZER_PATH)  # Sauvegarde le vectoriseur

# Entraîner le modèle
def train_model(data: pd.DataFrame):
    """Entraîne un modèle Random Forest avec un TfidfVectorizer."""
    vectorizer = TfidfVectorizer(max_features=5000)  # Initialise le vectoriseur TF-IDF
    model = RandomForestClassifier(random_state=42, class_weight='balanced')  # Initialise le modèle avec des poids équilibrés
    messages = data['message'].astype(str).apply(clean_text)  # Nettoie les messages
    labels = data['label'].astype(int)  # Convertit les étiquettes en entiers
    X = vectorizer.fit_transform(messages)  # Transforme les messages en vecteurs TF-IDF
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)  # Divise les données en ensembles d'entraînement et de test

    # Calculer les poids des échantillons
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)  # Calcule les poids pour équilibrer les classes

    # Optimisation des hyperparamètres
    param_grid = {
        'n_estimators': [100, 200, 300],  # Nombre d'arbres dans la forêt
        'max_depth': [5, 10, 15],  # Profondeur maximale des arbres
    }
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1', n_jobs=-1)  # Recherche sur grille avec validation croisée
    grid_search.fit(X_train, y_train)  # Entraîne le modèle avec les hyperparamètres optimaux

    # Extraire les résultats du GridSearch
    results = pd.DataFrame(grid_search.cv_results_)  # Convertit les résultats en DataFrame

    # Visualisation des résultats du GridSearch
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=results['param_n_estimators'], y=results['mean_test_score'], label='Mean F1 Score')  # Trace la courbe des scores
    plt.title('Résultats de la Recherche de Grille')
    plt.xlabel("Nombre d'Estimateurs")
    plt.ylabel('Score F1 Moyen')
    plt.legend()
    grid_search_path = os.path.join(OUTPUT_DIR, 'grid_search_results.png')  # Chemin pour sauvegarder la figure
    plt.savefig(grid_search_path)
    plt.close()

    model = grid_search.best_estimator_  # Récupère le meilleur modèle
    model.fit(X_train, y_train)  # Entraîne le meilleur modèle
    save_model_and_vectorizer(model, vectorizer)  # Sauvegarde le modèle et le vectoriseur

    # Évaluation sur les données de test
    y_pred = model.predict(X_test)  # Prédictions sur l'ensemble de test
    y_proba = model.predict_proba(X_test)[:, 1]  # Probabilités prédictives pour la classe positive

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)  # Calcule la courbe ROC
    roc_auc = auc(fpr, tpr)  # Calcule l'aire sous la courbe ROC

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Courbe ROC (aire = {roc_auc:.2f})')  # Trace la courbe ROC
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Trace la diagonale
    plt.xlabel('Taux de Faux Positifs')
    plt.ylabel('Taux de Vrais Positifs')
    plt.title('Courbe ROC (Receiver Operating Characteristic)')
    plt.legend(loc="lower right")
    roc_path = os.path.join(OUTPUT_DIR, 'roc_curve.png')  # Chemin pour sauvegarder la figure
    plt.savefig(roc_path)
    plt.close()

    st.subheader("Évaluation sur les données de test")
    st.text(classification_report(y_test, y_pred))  # Affiche le rapport de classification
    st.image(roc_path, caption="Courbe ROC")  # Affiche la courbe ROC
    st.image(grid_search_path, caption="Résultats de la Recherche de Grille")  # Affiche les résultats du GridSearch

    return model, vectorizer

# Générer des métriques
def generate_metrics(data: pd.DataFrame, model, vectorizer):
    """Génère les métriques de performance et une matrice de confusion."""
    messages = data['message'].astype(str).apply(clean_text)  # Nettoie les messages
    X = vectorizer.transform(messages)  # Transforme les messages en vecteurs TF-IDF
    y_true = data['label'].astype(int)  # Étiquettes réelles
    y_pred = model.predict(X)  # Prédictions du modèle

    report = classification_report(y_true, y_pred, output_dict=True)  # Génère le rapport de classification
    conf_matrix = confusion_matrix(y_true, y_pred)  # Génère la matrice de confusion

    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Spam', 'Spam'], yticklabels=['Non-Spam', 'Spam'])  # Trace la matrice de confusion
    plt.title('Matrice de Confusion (Confusion Matrix)')
    plt.xlabel('Prédit')
    plt.ylabel('Réel')
    confusion_path = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')  # Chemin pour sauvegarder la figure
    plt.savefig(confusion_path)
    plt.close()

    return report, confusion_path

# Interface utilisateur Streamlit
st.title("Analyseur SMS Spam - Classificateur avec Visualisation")  # Titre de l'application

# Réentraîner le modèle
st.header("Réentraîner le Modèle")  # En-tête pour la section de réentraînement
uploaded_file = st.file_uploader("Choisissez un fichier CSV", type=["csv"])  # Téléchargement d'un fichier CSV
if uploaded_file:
    try:
        data = pd.read_csv(uploaded_file, delimiter='\t', header=None, names=['label', 'message'])  # Charge les données
        data['label'] = data['label'].map({'ham': 0, 'spam': 1})  # Mappe les étiquettes de texte à des valeurs numériques
        model, vectorizer = train_model(data)  # Entraîne le modèle
        st.success("Modèle réentraîné avec succès.")  # Affiche un message de succès

        report, confusion_path = generate_metrics(data, model, vectorizer)  # Génère les métriques
        st.subheader("Matrice de Confusion")
        st.image(confusion_path)  # Affiche la matrice de confusion
        st.subheader("Métriques de Performance")
        st.write(pd.DataFrame(report).transpose())  # Affiche les métriques sous forme de tableau
    except Exception as e:
        st.error(f"Erreur : {e}")  # Affiche une erreur en cas de problème

# Charger modèle et vectoriseur
model, vectorizer = load_model_and_vectorizer()  # Charge le modèle et le vectoriseur

# Section de prédiction
st.header("Prédire un SMS")  # En-tête pour la section de prédiction
user_input = st.text_area("Entrez un SMS à analyser :", height=100)  # Champ de saisie pour le texte à prédire

if st.button("Classer le SMS"):
    if not model or not vectorizer:
        st.warning("Modèle non disponible. Veuillez réentraîner le modèle.")  # Avertissement si le modèle est manquant
    else:
        input_vectorized = vectorizer.transform([clean_text(user_input)])  # Transforme le texte utilisateur en vecteur TF-IDF
        prediction = model.predict(input_vectorized)[0]  # Prédit la classe du SMS
        st.success(f"Ce SMS est classé comme : **{'Spam' if prediction == 1 else 'Ham (Non-Spam)'}**")  # Affiche le résultat de la prédiction
